# Оглавление
- [Вступление](#intro)
- [Общая информация](#general_info)
- [Разбитие по выборкам](#splitting)
- [Исследование моделей](#exploring)
  - [Древо решения](#tree)
  - [Случайный лес](#forest)
  - [Логистическая регрессия](#log_regression)
  - [Выводы по исследованию](#exploring_conclusions)
- [Проверка на тестовой выборке](#test)
- [Проверка на адекватность](#sanity_check)
- [Вывод](#conclusion)

<a id='intro'></a>
## Вступление 
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

Имеем в наличии данные предоставленные оператором мобильной связи "Мегалайн" включающие в себя пользователей которые перешли на новые тарифные планы "Смарт" и "Ультра". На основе этих данных создадим модель машинного обучения которая поможет таргетировать рекомендации о переходе на новые тарифы для пользователей которые пользуются архивными тарифами.

Перед нами стоит задача получить accuracy не менее 0.75 на тестовой выборке.

<a id='general_info'></a>
## Откройте и изучите файл

**Импортируем необходимые библиотеки**

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

**Читаем исходный файл**

In [2]:
users = pd.read_csv('users_behavior.csv')

**Основная информация о датафрейме**

In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
users.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
users.groupby('is_ultra')['is_ultra'].agg('count') / len(users) * 100

is_ultra
0    69.352831
1    30.647169
Name: is_ultra, dtype: float64

В предоставленных данных есть сведения о 3214 пользователей пользующихся новыми тарифными планами. Данные уже предобработаны, поэтому можем приступать к разборке их на выборки. 69% пользователей используют тарифный план "Смарт" и 31% - "Ультра"

<a id='splitting'></a>
## Разобьём данные на выборки

Так как это все данные что у нас есть в наличии, нам нужно разбить их на три выборки:
- **Тренировачная**: нужна для обучения модели
- **Валидационная**: поможет выявить переобучение модели
- **Тестовая**: нужна для оценки готовой модели

Поделим в соотношении 3:1:1, тоесть 60% - тренировочная выборка и по 20% валидационная и тестовая выборки.

Определим целевой признак и признаки. Целевой признак в данном датасете выберем столбец `is_ultra` содержащий информацию о том каким тарифом пользуется пользователь, "Смарт" (значение 0) или "Ультра" (значение 1).

In [6]:
features = users.drop('is_ultra', axis=1,)
target = users['is_ultra']

Начнём разибивать данные на выборки.

In [7]:
features_train, features_rest, target_train, target_rest = train_test_split(
    features, target, train_size=0.6, random_state=12345, stratify=target)

In [8]:
print(features_train.shape)
print(features_rest.shape)
print(target_train.shape)
print(target_rest.shape)

(1928, 4)
(1286, 4)
(1928,)
(1286,)


Разбили датасет на две части - тренировочная выборка и остальные данные. Проверили размеры после разибивки, разибка выполнена корректно. Теперь остальные разобьём пополам и получим наши валидационные и тестовые выборки

In [9]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_rest, target_rest, test_size=0.5, random_state=12345, stratify=target_rest)

In [10]:
print(features_valid.shape)
print(features_test.shape)
print(target_valid.shape)
print(target_test.shape)

(643, 4)
(643, 4)
(643,)
(643,)


Получили валидационные и тестовые выборки, проверили размеры, разбивка выполнена правильно. После нахождения лучших гиперпараметров можем обучить полученную модель используя тренировочную+валидационную выборки, для этого склеим эти выборки для такой ситуации.

In [11]:
features_train_val = pd.concat([features_train, features_valid])
target_train_val = pd.concat([target_train, target_valid])

Сделаем ещё одну разбивку. Будем использовать кросс-валидацию для нахождения лучших гиперпараметров, нам не нужно делать заранее валидационную выборку, поэтому сделаем разбивку на две части: тренировочные данные(75%) и тестовые(25%).

In [12]:
features_cv_train, features_cv_test, target_cv_train, target_cv_test = train_test_split(
    features, target, test_size=0.25, random_state=12345, stratify=target)

In [13]:
print(features_cv_train.shape)
print(features_cv_test.shape)
print(target_cv_train.shape)
print(target_cv_test.shape)

(2410, 4)
(804, 4)
(2410,)
(804,)


Разбивка выполнена правильно. Можем приступать к исследованию моделей.

<a id='exploring'></a>
## Исследуем модели

Целевой признак категориальный, поэтому будем стоить модели классификации. Построим следующие модели: дерево решения, случайный лес и логистическую регрессию. И впоследствии выберем лучшую из них.
<a id='tree'></a>
### Древо решения 

С помощью цикла переберем различные гиперпараметры, обучим модель на тренировочных данных и оценим качество на валидацинной выборке.

In [14]:
%%time

best_tree_model = None
best_tree_result = 0
for depth in range(1, 11):
    for leafs in range(2, 51):
        for samp_leafs in range(1, 51):
            model = DecisionTreeClassifier(random_state=12345,
                                           max_depth=depth,
                                           max_leaf_nodes,
                                           min_samples_leaf=samp_leafs)
            model.fit(features_train, target_train)
            result = model.score(features_valid, target_valid)
            if result > best_tree_result:
                best_tree_model = model
                best_tree_result = result

print("Качество лучшей модели на валидационной выборке:", best_tree_result)
print("Лучшая модель:", best_tree_model)

Качество лучшей модели на валидационной выборке: 0.8087091757387247
Лучшая модель: DecisionTreeClassifier(max_depth=8, max_leaf_nodes=42, min_samples_leaf=7,
                       random_state=12345)
CPU times: user 2min 11s, sys: 300 ms, total: 2min 11s
Wall time: 2min 11s


Лучший результат на валидационной сборке, **качество - 0.809**, показала модель со следующими гиперпараметрами:
- max_depth = 8
- max_leaf_nodes=42
- min_samples_leaf=7
- random_state задали для того чтобы зафиксировать псевдослучайность

<a id='forest'></a>
### Случайный лес

С помощью цикла переберем различные гиперпараметры, обучим модель на тренировочных данных и оценим качество на валидацинной выборке.

In [15]:
%%time

best_forest_model = None
best_forest_result = 0
for est in range(10, 151, 10):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345,
                                       n_estimators=est,
                                       max_depth=depth)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_forest_result:
            best_forest_model = model
            best_forest_result = result

print("Качество лучшей модели на валидационной выборке:", best_forest_result)
print("Лучшая модель:", best_forest_model)

Качество лучшей модели на валидационной выборке: 0.8211508553654744
Лучшая модель: RandomForestClassifier(max_depth=9, n_estimators=40, random_state=12345)
CPU times: user 28.5 s, sys: 156 ms, total: 28.7 s
Wall time: 28.7 s


Лучший результат на валидационной сборке, **качество - 0.821**, показала модель со следующими гиперпараметрами:
- max_depth = 9
- n_estimators = 40
- random_state задали для того чтобы зафиксировать псевдослучайность

Пробовали добавлять в цикл перебор следущие гиперпараметры max_leaf_nodes, min_samples_leaf. Подбор гиперпаметров занимал намного больше времени и результат в большинстве случаев был хуже, поэтому их оставили по умолчанию.

Попробуем использовать кросс-валидацию GridSearchCV из библиотеки sklearn для нахождения лучших гиперпараметров для случайного леса. Сначала подготовим гиперпараметры которые мы хотим перебрать. Создадим пустые списки и заполним их с помощью циклов значениями которые будем перебирать.

In [16]:
n_estimators = []
max_depth = []

for est in range(10, 151, 10):
    n_estimators.append(est)
for depth in range(1, 11):
    max_depth.append(depth)

Теперь укажем какую модель и какие параметры мы будем загружать в GridSearchCV.

In [17]:
forest = RandomForestClassifier()
params = {
    'random_state': [12345],
    'n_estimators': n_estimators,
    'max_depth': max_depth
         }

forest_clf = GridSearchCV(
    estimator=forest,
    param_grid=params,
    cv=5,
    scoring='accuracy'
)

Можем приступать к обучению модели, для GridSearchCV используем данные которые специально для этого разбивали.

In [18]:
%%time
forest_clf.fit(features_cv_train, target_cv_train)

CPU times: user 2min 21s, sys: 876 ms, total: 2min 22s
Wall time: 2min 22s


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                          100, 110, 120, 130, 140, 150],
                         'random_state': [12345]},
             scoring='accuracy')

Посмотрим на лучшую модель после проверки кросс-валидацией.

In [19]:
print(forest_clf.best_estimator_)
print("Качество модели:", forest_clf.best_score_)

RandomForestClassifier(max_depth=8, n_estimators=10, random_state=12345)
Качество модели: 0.8024896265560166


Получили **качество модели: 0.802**.

<a id='log_regression'></a>
### Логистическая регрессия 

Попробуем различные гиперпараметры, у логистической регрессии малое количество гиперпараметров и переобучение модели ей не грозит. Обучим модель на тренировочных данных и оценим качество на валидацинной выборке.

In [20]:
log_reg_model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000, multi_class='ovr')
log_reg_model.fit(features_train, target_train)
log_reg_model.score(features_valid, target_valid)

0.7387247278382582

Лучший результат на валидационной сборке, **качество - 0.756**, показала модель со следующими гиперпараметрами:
- solver = 'lbfgs'
- max_iter = 1000
- multi_class='ovr'
- random_state задали для того чтобы зафиксировать псевдослучайность

Пробовали изменять solver на  другие варианты такие как 'newton-cg', 'liblinear', 'sag', 'saga', но 'lbfgs' показал себя лучше всех. Больше всего на результат повлияло указание гиперпараметра multi_class. При попытках использовать 'multinomial' результат был хуже чем 'ovr'.
<a id='exploring_conclusions'></a>
### Выводы по исследованию моделей 

Построили следующие модели классификации: Древо Решений, Случайный Лес и Логистическую Регрессию. Обучили их на тренировочных данных. Пробовали различные гиперпараметры:
- Для древа решений сделали цикл с помощью которого перебирали следующие гиперпараметры: max_depth, max_leaf_nodes, min_samples_leaf. Результат работы модели оценивали на валидационной выборке с помощью метрики качества accuracy. Таким образом удалось достичь **accuracy - 0.809**.
- Для случайного леса сделали цикл с помощью которого перебирали следующие гиперпараметры: max_depth, n_estimators. Пробовали так же изменять max_leaf_nodes и min_samples_leaf, но результат при оставлении их по умолчанию лучше. Таким образом удалось достичь **accuracy - 0.821**. Так же попробовали использовать кросс-валидацию, разбивка данных происходила по другому поэтому полученную **метрику качества 0.802** нет смысла сравнивать с другими, построенными тут моделями.
- У логистической мало гиперпараметров которыми мы может повлиять на результат, лучший результат удалось достить указав multi_class='ovr'. **Accuracy** такой модели - **0.739**

**Лучшая модель - Случайный Лес, accuracy - 0.821 на валидационной выборке.**

<a id='test'></a>
## Проверим модель на тестовой выборке

Дообучим нашу лучшую модель на тренировочной + валидационной выборке.

In [21]:
best_forest_model.fit(features_train_val, target_train_val)

RandomForestClassifier(max_depth=9, n_estimators=40, random_state=12345)

Проверим как эта модель поведёт себя на тестовой выборке.

In [22]:
best_forest_model.score(features_test, target_test)

0.8289269051321928

На тестовой выборке accuracy лучше, **0.829** против 0.821 на валидационной.

Мы ранее строили модель используя кросс-валидацию, проверим её на тестовой выборке.

In [23]:
forest_clf.score(features_cv_test, target_cv_test)

0.8121890547263682

Результат **0.812**

### Выводы по тестовой выборке

Проверили лучшую модель на тестовой выборке. **Accuracy 0.829** нас устраивает. Теперь проверим модель на адекватность.

<a id='sanity_check'></a>
## Проверим модели на адекватность

Сравним нашу лучшую модель с "глупой" моделью которая будет игнорировать входящие признаки и делать предсказание случайным образом.
Попробуем два варианта гиперпараметра strategy у этой модели - most_frequent и stratified

In [24]:
dummy = DummyClassifier(random_state=12345, strategy="most_frequent")
dummy.fit(features_train, target_train)
print('Тренировочная выборка:', dummy.score(features_train, target_train))
print('Валидационная выборка:', dummy.score(features_valid, target_valid))
print('Тестовая выборка:', dummy.score(features_test, target_test))

Тренировочная выборка: 0.6934647302904564
Валидационная выборка: 0.6936236391912908
Тестовая выборка: 0.6936236391912908


In [25]:
dummy = DummyClassifier(random_state=12345, strategy="stratified")
dummy.fit(features_train, target_train)
print('Тренировочная выборка:', dummy.score(features_train, target_train))
print('Валидационная выборка:', dummy.score(features_valid, target_valid))
print('Тестовая выборка:', dummy.score(features_test, target_test))

Тренировочная выборка: 0.5809128630705395
Валидационная выборка: 0.5598755832037325
Тестовая выборка: 0.5567651632970451


А теперь посмотрим посмотрим на нашу лучшую модель на трех выборках.

In [26]:
print('Тренировочная выборка:', best_forest_model.score(features_train, target_train))
print('Валидационная выборка:', best_forest_model.score(features_valid, target_valid))
print('Тестовая выборка:', best_forest_model.score(features_test, target_test))

Тренировочная выборка: 0.8724066390041494
Валидационная выборка: 0.8880248833592534
Тестовая выборка: 0.8289269051321928


На всех выборках случайный лес показал значительно лучшие результаты чем случайная модель. Можем сделать вывод что модель адекватна.

<a id='conclusion'></a>
## Вывод 

Перед нами стояла задача классификации с известным целевым признаком. Исследовали три модели: древо решения, случайный лес и логистическую регрессию. По итогу исследования нашли наилучшую модель - случайный лес, где смогли добится accuracy = 0.821 на валидационной сборке. Проверили её на тестовых данных и получили результат 0.829. Сравнили модель со случайной и убедились в адекватности случайного леса. Задача стояла получить модель с accuracy больше 0.75 чего мы смогли добится.